In [1]:
%pip install scikit-multilearn

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

2022-07-11 21:46:10.062470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: /usr/lib/x86_64-linux-gnu/libcuda.so.1: file too short; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-07-11 21:46:10.062491: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-11 21:46:10.062506: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-07-11 21:46:10.062650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [4]:
tf.__version__

'2.9.1'

In [5]:
paths = ['/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            'neurons_1' : 32, 
            'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    test_results, ba = har.run()
    #model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
print(max(bas))


X train-test shape: (3168, 226) - X test shape: (792, 226)
y train-test shape: (3168, 6) - y test shape: (792, 6)
Epoch 1/40
254/254 [==============================] - 1s 1ms/step - loss: 1.0344 - val_loss: 1.0746
Epoch 2/40
254/254 [==============================] - 0s 774us/step - loss: 1.0081 - val_loss: 1.0420
Epoch 3/40
254/254 [==============================] - 0s 850us/step - loss: 0.9953 - val_loss: 1.0638
Epoch 4/40
254/254 [==============================] - 0s 823us/step - loss: 0.9971 - val_loss: 1.0435
Epoch 5/40
254/254 [==============================] - 0s 808us/step - loss: 1.0007 - val_loss: 1.0394
Epoch 6/40
254/254 [==============================] - 0s 865us/step - loss: 1.0015 - val_loss: 1.0412
Epoch 7/40
254/254 [==============================] - 0s 853us/step - loss: 0.9955 - val_loss: 1.0417
Epoch 8/40
254/254 [==============================] - 0s 822us/step - loss: 1.0002 - val_loss: 1.0443
Epoch 9/40
254/254 [==============================] - 0s 763us/step - lo

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [7]:
'''def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'''

'def nan_mse(y_actual, y_predicted):\n    stack = tf.stack((tf.math.is_nan(y_actual), \n                      tf.math.is_nan(y_predicted)),\n                     axis=1)\n    is_nans = tf.keras.backend.any(stack, axis=1)\n    per_instance = tf.where(is_nans,\n                            tf.zeros_like(y_actual),\n                            tf.square(tf.subtract(y_predicted, y_actual)))\n    print(stack)\n    print(per_instance)\n    return tf.reduce_mean(per_instance, axis=0)\n\nprint(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'

In [8]:
paths = ['/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)

Trial 76 Complete [00h 00m 11s]
val_avg_multilabel_BA_2: 0.9099557995796204

Best val_avg_multilabel_BA_2 So Far: 0.9160040020942688
Total elapsed time: 00h 00m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
51/51 [==============================] - 1s 6ms/step - loss: 0.3988 - avg_multilabel_BA_2: 0.9096 - val_loss: 0.1177 - val_avg_multilabel_BA_2: 0.9096
Epoch 2/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1009 - avg_multilabel_BA_2: 0.9096 - val_loss: 0.1063 - val_avg_multilabel_BA_2: 0.9097
Epoch 3/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0895 - avg_multilabel_BA_2: 0.9097 - val_loss: 0.0992 - val_avg_multilabel_BA_2: 0.9098
Epoch 4/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0818 - avg_multilabel_BA_2: 0.9099 - val_loss: 0.1066 - val_avg_multilabel_BA_2: 0.9100
Epoch 5/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0776 - avg_multilabel_BA_2: 0.9101 - val_loss: 0.0838 - val_avg_multil

In [10]:

bas = []
config = {
        #'df_path': path,
        'df_path': './sample_data/exp_/fold_0/raw_40.csv',
        'neurons_1' : 32, 
        'neurons_2' : 16, 
        'labels': labels
}

har = utils.HAR(config)
test_results, ba = har.run()
#model, best_hps, best_epoch, test_results, ba = har.hypertunning()
bas.append(ba.numpy())


print(bas)
print(max(bas))

X train-test shape: (242349, 226) - X test shape: (60588, 226)
y train-test shape: (242349, 6) - y test shape: (60588, 6)
Epoch 1/40
19388/19388 [==============================] - 16s 794us/step - loss: 51.7867 - val_loss: 32.7636
Epoch 2/40
19388/19388 [==============================] - 15s 791us/step - loss: 138.8578 - val_loss: 145.2188
Epoch 3/40
19388/19388 [==============================] - 15s 793us/step - loss: 222.4855 - val_loss: 149.7679
Epoch 4/40
19388/19388 [==============================] - 15s 795us/step - loss: 305.4887 - val_loss: 309.1640
Epoch 5/40
19388/19388 [==============================] - 15s 789us/step - loss: 387.3051 - val_loss: 681.7822
Epoch 6/40
19388/19388 [==============================] - 15s 796us/step - loss: 468.6788 - val_loss: 288.7400
Epoch 7/40
19388/19388 [==============================] - 16s 800us/step - loss: 548.9426 - val_loss: 566.7703
Epoch 8/40
19388/19388 [==============================] - 15s 799us/step - loss: 629.5294 - val_loss: 5

In [ ]:
har.data.x.head()

NameError: name 'har' is not defined

In [ ]:

import glob
bas = []
folderpath = '/home/wander/OtherProjects/har_flower/sample_data'
for path in glob.iglob(f'{folderpath}/**.csv', recursive=True):
    config = {
            'df_path': path,
            #'df_path': './sample_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16,
            #'from_saved': ../full_data/exp_/saved_model_fold_0,
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
    

[]


In [ ]:
har.mlp.model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 56)                12712     
                                                                 
 dense_37 (Dense)            (None, 8)                 456       
                                                                 
 dense_38 (Dense)            (None, 6)                 54        
                                                                 
Total params: 13,222
Trainable params: 13,222
Non-trainable params: 0
_________________________________________________________________


In [ ]:
har.mlp.data.x_train

AttributeError: 'MLPMultilabel' object has no attribute 'data'

In [ ]:

import glob
bas = []
path = '/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv'
config = {
        'df_path': path,
        #'df_path': './sample_data/exp_/fold_0/raw_40.csv',
        #'neurons_1' : 32, 
        #'neurons_2' : 16,
        'from_saved': '../full_data/exp_/saved_model_fold_0',
        'labels': labels
}

har = utils.HAR(config)
#test_results, ba = har.run()
#model, best_hps, best_epoch, test_results, ba = har.hypertunning()
#bas.append(ba.numpy())


#print(bas)
har.mlp.model.summary()  

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 44)                9988      
                                                                 
 dense_7 (Dense)             (None, 6)                 270       
                                                                 
Total params: 10,258
Trainable params: 10,258
Non-trainable params: 0
_________________________________________________________________
